In [29]:
import pandas as pd
from seml import get_results

In [ ]:
df = get_results(
    'dp_timeseries_traditional_baselines_dp_eval',
    [
        'config.seed',
        'config.predictor_name',
        'config.neighboring_relation.target_sensitivity',
        'config.budget_epsilon',
        'config.budget_delta',
        'result.metrics_test.mean_wQuantileLoss',
        'result.metrics_test.MASE'
    ],
    to_data_frame=True,
    filter_dict={
        'config.dataset_kwargs.dataset_name': 'electricity',
    }
)

In [31]:
columns = df.columns[1:]

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [ ]:
df

In [33]:
def create_table(df: pd.DataFrame,
                 budget_delta: int,
                 target_sensitivity: float,
                 errors: bool = False
) -> pd.DataFrame:

    df = df.copy()

    df = df[
        (df['budget_delta'] == budget_delta)
        & (df['target_sensitivity'] == target_sensitivity)
    ]

    print(len(df))

    # Average over random seeds
    if not errors:
        df = df.groupby(
            ['predictor_name', 'budget_delta', 'budget_epsilon', 'target_sensitivity']
        ).mean().reset_index()
    else:
        df = df.groupby(
            ['predictor_name', 'budget_delta', 'budget_epsilon', 'target_sensitivity']
        ).std().reset_index()

    df = df.drop(columns=[
        '_id',
        'seed', 'budget_delta',
        'target_sensitivity', '_id'])

    df = df.sort_values(by=['predictor_name'])

    return df

In [34]:
budget_delta=0.0001
target_sensitivity=10.0

### No subsampling

In [ ]:
df_mean_no_subsampling = create_table(df, budget_delta=budget_delta, target_sensitivity=target_sensitivity)
df_mean_no_subsampling

In [ ]:
df_std_no_subsampling = create_table(df, budget_delta=budget_delta, target_sensitivity=target_sensitivity, errors=True)
df_std_no_subsampling

In [ ]:
i = 0
names = ['Seasonal ($r=1.0$)']

acc = f'{names[0]} &'

for i, (mean, std) in enumerate(zip(df_mean_no_subsampling['mean_wQuantileLoss'], df_std_no_subsampling['mean_wQuantileLoss'])):

    if (i > 0) and ((i % 5)  == 0):
        acc += ' ? \\\\'
        print(acc)
        acc = f'{names[i // 5]} &'
    acc += ' $' + f'{mean:.3f}' + '$ '
    acc += '\\tiny{$\\pm ' + f'{std:.3f}' + '$} &'

acc += ' ? \\\\ '
print(acc)